# Recommander systems



In [1]:
import numpy as np
from scipy import sparse
from scipy.sparse.linalg import svds
rho=0.3

## Question 1.1

We will run the file u.data where the data is 

In [2]:
def load_movielens(filename, minidata=False):
    """
    Cette fonction lit le fichier filename de la base de donnees
    Movielens, par exemple 
    filename = '~/datasets/ml-100k/u.data'
    Elle retourne 
    R : une matrice utilisateur-item contenant les scores
    mask : une matrice valant 1 si il y a un score et 0 sinon
    """

    data = np.loadtxt(filename, dtype=int)

    R = sparse.coo_matrix((data[:, 2], (data[:, 0]-1, data[:, 1]-1)),
                          dtype=float)
    R = R.toarray()  # not optimized for big data

    # code la fonction 1_K
    mask = sparse.coo_matrix((np.ones(data[:, 2].shape),
                              (data[:, 0]-1, data[:, 1]-1)), dtype=bool )
    mask = mask.toarray()  # not optimized for big data

    if minidata is True:
        R = R[0:100, 0:200].copy()
        mask = mask[0:100, 0:200].copy()

    return R, mask

print(load_movielens('u.data'))
R=load_movielens('u.data')[0]
mask=load_movielens('u.data')[1]
R

(array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]]), array([[ True,  True,  True, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [ True, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False,  True, False, ..., False, False, False]]))


array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

The size of the matrix R obtained is :

In [3]:
R.shape

(943, 1682)

So the size is 943x1682 indeed. Besides the option _minidata_ allows to consider a smaller set of data (that is to say the rating of the first hundred users) if it is set to _True_ (otherwise we consider the whole set of data).

## Question 1.2

According to the precedent result : there are 943 users (the number of lines in the matrix R with the whole dataset) and 1682 movies (number of columns in the matrix R).

Thus, the total number of grades is at least 943x1682 that is to say 1 586 126 (it is probably less given the fact that not every users have watch and rated all the movies)

In [4]:
Q0, _, P0 = svds(R,k=4)

## Question 2.1

We have :
    
$$ g(P)=\frac{1}{2} ||1_{K} \circ (R-Q^{0}P)||_{F}^{2} + \frac{\rho}{2}||Q^{0}||_{F}^{2}+\frac{\rho}{2}||P||_{F}^{2}$$
$$=\frac{1}{2} \sum_{u \in U} \sum_{i \in I} 1_{K_{(u,i)}} (R_{u,i}-\sum_{f \in F} Q_{u,f}^{0} P_{f,i})^{2} +\frac{\rho}{2}||Q^{0}||_{F}^{2}+\frac{\rho}{2} \sum_{i \in U} \sum_{u \in I} P_{u,i}^{2}$$

Thus, we get :

$$\frac{\partial g}{\partial P_{k,l}} = \sum_{u \in U} \sum_{i \in I} 1_{K_{(u,i)}} \delta_{i,l} (R_{u,i}-\sum_{f \in F} Q_{u,f}^{0} P_{f,i}) x (-Q_{u,l}^{0})+ \rho P_{k,l}$$
$$=\sum_{u \in U} 1_{K_{(u,l)}}(R_{u,l}-\sum_{f \in F} Q_{u,f}^{0} P_{f,l}) x (-Q_{u,l}^{0})+ \rho P_{k,l}$$

And by switching to matrix format, we finally obtain :

$$\nabla g(P)=(Q^{0})^{T}(1_{K} \circ (Q^{0}P-R))+\rho P$$

## Question 2.2

We simply compute the expression we found previously.

In [5]:
def objective(P, Q0, R, mask, rho):
    """
    La fonction objectif du probleme simplifie.
    Prend en entree 
    P : la variable matricielle de taille C x I
    Q0 : une matrice de taille U x C
    R : une matrice de taille U x I
    mask : une matrice 0-1 de taille U x I
    rho : un reel positif ou nul

    Sorties :
    val : la valeur de la fonction
    grad_P : le gradient par rapport a P
    """

    tmp = (R - Q0.dot(P)) * mask

    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q0 ** 2) + np.sum(P ** 2))

    grad_P = np.transpose(Q0).dot((Q0.dot(P)-R)*mask) + rho*P

    return val, grad_P


## Question 2.3

In [6]:
def gradient(g, P0, gamma, epsilon):
    Pk=P0
    while np.sqrt(np.sum(objective(Pk,Q0,R,mask,rho)[1]**2))>epsilon:
        Pk=Pk-gamma*objective(Pk,Q0,R,mask,rho)[1]
    return Pk

## Question 2.4

For the value of gamma, we take $\gamma = \frac{1}{L_{0}}=\frac{1}{\rho + ||Q_{0}^{T} Q_{0}||_{F}}$ (according to the theorem 3.2.1)

In [7]:
gamma=1/(rho+np.sqrt(np.sum(np.transpose(Q0).dot(Q0)**2)))
gradient(objective(P0, Q0, R, mask, rho),P0,gamma,1)

array([[-2.23284225e+00,  4.90664665e-01,  9.98919015e+00, ...,
        -4.74987005e-01,  3.92212474e-02,  8.67808467e-01],
       [ 4.53777371e+00, -1.35365298e+01, -2.56785789e+00, ...,
         3.63533351e-01, -3.15394841e-01, -1.77455812e-01],
       [-2.02978127e+01, -4.68180885e-01, -1.07452929e+01, ...,
        -3.43502581e-01,  8.21005494e-02,  1.54999171e-01],
       [ 5.76416726e+01,  2.77454348e+01,  1.98640643e+01, ...,
         6.09197523e-02,  6.75813808e-01,  6.32544121e-01]])

## Question 2.5

We add the line search into the algorithm. We just have to chose $L_{k}$ as stated in the lesson.

In [8]:
def gradient(g, P0, epsilon,a,b):
    Pk=P0
    while np.sqrt(np.sum(objective(Pk,Q0,R,mask,rho)[1]**2))>epsilon:
        l=0
        while (objective(Pk-(b*(a**l))*objective(Pk,Q0,R,mask,rho)[1],Q0,R,mask,rho)[0])>(objective(Pk,Q0,R,mask,rho)[0]+np.trace(np.dot(objective(Pk,Q0,R,mask,rho)[1],np.transpose(Pk-(b*(a**l))*objective(Pk,Q0,R,mask,rho)[1]-Pk)))+(1/(2*b*(a**l)))*np.sum((Pk-Pk-(b*(a**l))*objective(Pk,Q0,R,mask,rho)[1])**2)):
            l+=1
        Pk=Pk-(b*(a**l))*objective(Pk,Q0,R,mask,rho)[1]
    return Pk

gradient(objective(P0, Q0, R, mask, rho),P0,1,0.5,1)

array([[-2.23398105e+00,  4.90322723e-01,  9.98772528e+00, ...,
        -4.74423899e-01,  3.91619850e-02,  8.66849934e-01],
       [ 4.53883470e+00, -1.35332793e+01, -2.56588860e+00, ...,
         3.63102008e-01, -3.14917972e-01, -1.77259625e-01],
       [-2.02984823e+01, -4.65096299e-01, -1.07474683e+01, ...,
        -3.43094903e-01,  8.19763912e-02,  1.54827764e-01],
       [ 5.76420231e+01,  2.77486548e+01,  1.98678781e+01, ...,
         6.08473595e-02,  6.74790793e-01,  6.31843664e-01]])

## Question 3.1

Given the fact that f is polynomial of degree four, its gradient is of degree three and can be written as follows :

$$\nabla f (P,Q) = \sum_{0\leq i \leq 3} \sum_{0\leq j \leq 3} a_{i} a_{j} P^{i}Q^{j}$$

Let us suppose that $\nabla f $ i Lipschitz continuous. Then $\exists L, \forall (A,B),(C,D), ||\nabla f (A,B)-\nabla f (C,D)|| \leq ||(A,B)-(C,D)||$. Let us fix $L$ and take $(A,B)=(\alpha I_{n},\alpha I_{n})$ for $\alpha >1$ and (C,D)=(I_{n},I_{n}). Hence we have :

$$||\nabla f (A,B)-\nabla f (C,D)||=||\sum_{0\leq i \leq 3} \sum_{0 \leq j \leq 3} a_{i} a_{j} (\alpha^{i+j} -1)I_{n}||=|\sum_{0\leq i \leq 3} \sum_{0 \leq j \leq 3} a_{i} a_{j} (\alpha^{i+j} -1)| \times n$$

and :

$$||(A,B)-(C,D)||=|\alpha - 1|\times ||(I_{n},I_{n})|| =\sqrt{2}|\alpha -1|\times n$$

By Lipschitz continuity, we have $|\sum_{0\leq i \leq 3} \sum_{0 \leq j \leq 3} a_{i} a_{j} (\alpha^{i+j} -1)| \leq L \sqrt{2}|\alpha -1|$ that is to say $\frac{|\sum_{0\leq i \leq 3} \sum_{0 \leq j \leq 3} a_{i} a_{j} (\alpha^{i+j} -1)|}{|\alpha -1|} \leq \sqrt{2}L$. However $\lim_{\alpha \to +\infty} \frac{|\sum_{0\leq i \leq 3} \sum_{0 \leq j \leq 3} a_{i} a_{j} (\alpha^{i+j} -1)|}{|\alpha -1|} = +\infty$ obviously (the degree of the numerator for $\alpha$ is at least 3) so for $\alpha$ great enough, $\frac{|\sum_{0\leq i \leq 3} \sum_{0 \leq j \leq 3} a_{i} a_{j} (\alpha^{i+j} -1)|}{|\alpha -1|}> \sqrt{2}L$, leading to an absurdity.

So $\nabla f$ is not Lipschitz continuous.

## Question 3.2



In [9]:
def total_objective(P, Q, R, mask, rho):
    """
    La fonction objectif du probleme complet.
    Prend en entree 
    P : la variable matricielle de taille C x I
    Q : la variable matricielle de taille U x C
    R : une matrice de taille U x I
    mask : une matrice 0-1 de taille U x I
    rho : un reel positif ou nul

    Sorties :
    val : la valeur de la fonction
    grad_P : le gradient par rapport a P
    grad_Q : le gradient par rapport a Q
    """

    tmp = (R - Q.dot(P)) * mask

    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q ** 2) + np.sum(P ** 2))

    grad_P = np.transpose(Q).dot((Q.dot(P)-R)*mask) + rho*P

    grad_Q =((Q.dot(P)-R)*mask).dot(np.transpose(P)) + rho*Q

    return val, grad_P, grad_Q


def gradient(g, P0, Q0, epsilon,a,b):
    Pk=P0
    Qk=Q0
    while np.sum(total_objective(Pk,Qk,R,mask,rho)[1]**2)+np.sum(total_objective(Pk,Qk,R,mask,rho)[2]**2)>epsilon**2:
        l=0
        while (total_objective(Pk-(b*(a**l))*total_objective(Pk,Qk,R,mask,rho)[1],Qk-(b*(a**l))*total_objective(Pk,Qk,R,mask,rho)[2],R,mask,rho)[0])>(total_objective(Pk,Qk,R,mask,rho)[0]+np.trace(np.dot(total_objective(Pk,Qk,R,mask,rho)[1],np.transpose(Pk-(b*(a**l))*total_objective(Pk,Qk,R,mask,rho)[1]-Pk)))+np.trace(np.dot(total_objective(Pk,Qk,R,mask,rho)[2],np.transpose(Qk-(b*(a**l))*total_objective(Pk,Qk,R,mask,rho)[2]-Qk)))+(1/(2*b*(a**l)))*(np.sum((Pk-Pk-(b*(a**l))*total_objective(Pk,Qk,R,mask,rho)[1])**2)+np.sum((Qk-Qk-(b*(a**l))*total_objective(Pk,Qk,R,mask,rho)[2])**2))):
            l+=1
        Pk=Pk-(b*(a**l))*total_objective(Pk,Qk,R,mask,rho)[1]
        Qk=Qk-(b*(a**l))*total_objective(Pk,Qk,R,mask,rho)[2]
    return (Pk,Qk)

P,Q=gradient(_,P0,Q0,100,0.5,0.01)
P
Q

array([[ 0.6832397 ,  0.36335231,  0.19912686,  2.37247675],
       [ 0.0447386 ,  0.58332141, -0.30943862,  2.06413352],
       [ 0.26708602,  0.43695545, -0.45284812,  1.63231321],
       ...,
       [ 0.13389931,  0.46086217, -0.70099297,  2.19289759],
       [-0.83558267, -0.45981307,  0.29659291,  2.52364109],
       [ 0.90664247, -0.24161946, -1.15339682,  2.06241541]])

The algorithm return two matrices P and Q. Their product represents the ratings users would most likely give to movies. More spectifically, for a user $u$, $Q_{.,u}$ represents the user in the feature space where the user-movie interaction is an inner product : it measures the extent of interest the user has in movies that are high on the corresponding factors. Likewise, $P_{i,.}$ represents the movie $i$ on this feature space : it measures the extent of interest the user has in the movie.

## Question 3.3



In [20]:
R_bis=np.dot(Q,P)
print(R_bis.shape)
maximum=0
movie=0
for k in range(len(R_bis[0])):
    if R_bis[299][k]>=maximum:
        maximum=R_bis[299][k]
        movie=k
print(movie)

(943, 1682)
271


We can conclude by saying we would recommand the movie n°271 to the user 300. 